# Background

This notebook contains all necessary steps for pre-processing and variant calling analysis. The objective for this notebook is to conduct variant calling analysis for a single gene of interest in both pre-treatment and during/post-treatment samples of a single patient in the study described in *Circulating tumor DNA sequencing in colorectal cancer patients treated with first-line chemotherapy with anti-EGFR*. The raw FASTQ files for this study are available on NCBI under BioProject PRJNA714799.

# Excercise Specific Information

Patient: CTC360

Pre-Treatment Sample Reads:
- Biosample - https://www.ncbi.nlm.nih.gov/biosample/18318230
- SRA - https://www.ncbi.nlm.nih.gov/sra?LinkName=biosample_sra&from_uid=18318230
- SRR - https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR13973704&display=metadata
- FASTQ - SRR13973704.fastq

During/Post-Treatment Sample Reads:
- Biosample -  https://www.ncbi.nlm.nih.gov/biosample/18318229
- SRA - https://www.ncbi.nlm.nih.gov/sra?LinkName=biosample_sra&from_uid=18318229
- SRR - https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR13973705&display=metadata
- FASTQ - SRR13973705.fastq

Gene of Interest: TP53
- Refseq - https://www.ncbi.nlm.nih.gov/gene/7157
- UCSC GB - https://genome.ucsc.edu/cgi-bin/hgGene?hgg_gene=ENST00000618944.4&hgg_chrom=chr17&hgg_start=7668401&hgg_end=7675493&hgg_type=knownGene&db=hg38
- Acts as a tumor suppressor in many tumor types; induces growth arrest or apoptosis depending on the physiological circumstances and cell type. Involved in cell cycle regulation as a trans-activator that acts to negatively regulate cell division by controlling a set of genes required for this process.
- 71 mutations over 64 patients (68.8%) at baseline in study.
- The TP53 gene is the most frequently mutated gene (>50%) in human cancer, indicating that the TP53 gene plays a crucial role in preventing cancer formation.

Reference Genome: hg38
- https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.26_GRCh38/GCF_000001405.26_GRCh38_genomic.fna.gz 

Known Variants VCF: dbSNP (latest release)
- https://ftp.ncbi.nih.gov/snp/latest_release/VCF/GCF_000001405.40.gz

Clinical Relevance Annotation: ClinVar
- https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20251109.vcf.gz


# Data Download

Download reference genome FASTA (hg38), known variants VCF (dbSNP), and clinical relevance annotation VCF (ClinVar).

In [ ]:
%%bash 
# Create data directory
mkdir data

# Navigate to data directory
cd data

# Reference genome
wget 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.26_GRCh38/GCF_000001405.26_GRCh38_genomic.fna.gz'
gunzip GCF_000001405.26_GRCh38_genomic.fna.gz
mv GCF_000001405.26_GRCh38_genomic.fna.gz hg38_ref.fa

# Known Variants
wget 'https://ftp.ncbi.nih.gov/snp/latest_release/VCF/GCF_000001405.40.gz'
gunzip GCF_000001405.40.gz

# Clinical Relevance Annotations
wget 'https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20251109.vcf.gz'

# Navigate to root directory for processing
cd ..

To download patient sample sequence read FASTQ files, follow specified above and click download FASTQ. Make sure to store FASTQ files in the data directory to avoid file path conflicts in pre-processing.

# Pre-Processing

Pre-processing of sequence reads in FASTQ files downloaded from SRA was conducted in adherence with *Best practices for variant calling in clinical sequencing* (https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00791-w) and GATK's best practice workflows for data pre-processing for variant discovery (https://gatk.broadinstitute.org/hc/en-us/articles/360035535912-Data-pre-processing-for-variant-discovery).

**Step 1: Generate index files for hg38 reference genome.**
- hg38 was used, instead of hg19 (used in study), since it contains the most comprehensive and up to date representation of the human reference genome.
- BWA index generates index file used for alignment with BWA MEM
- samtools faidx generates index file used by for base quality score recalibration


In [ ]:
%%bash
# Alignment
bwa index -p hg38 data/hg38_ref.fa

# Base quality score recalibration
samtools faidx data/hg38_ref.fa

**Step 2: Align sequence reads to hg38 reference genome.**
- BWA MEM outputs SAM
- Variant calling analysis using GATK requires BAM file input.
- To avoid saving large SAM file, pipe output from alignment straight into BAM conversion.
- '-Sb' specifies SAM input w/ BAM output
- Converted BAM files piped directly into samtools sort to generate sorted BAM file.

In [ ]:
%%bash
# Pre-treatment
bwa mem hg38 data/SRR13973704.fastq | samtools view -Sb | samtools sort > data/SRR13973704.bam

# During/post-treatment
bwa mem hg38 data/SRR13973705.fastq | samtools view -Sb | samtools sort > data/SRR13973705.bam

**Step 3: Index sorted BAM files**
- Necessary for filtering BAM by region.
- BAM filtering by region necessary to reduce uneccessary processing.

In [ ]:
%%bash
# Pre-treatment
samtools index data/SRR13973704.bam

# During/Post-treatment
samtools index data/SRR13973705.bam

**Step 4: Filter BAM by region**
- Only interested in TP53
- TP53 refseq coordinates - NC_000017.11:7668421-7687490

In [ ]:
%%bash
# Pre-treatment
samtools view -b data/SRR13973704.bam "NC_000017.11:7668421-7687490" > data/SRR13973704_tp53.bam

# During/post-treatment
samtools view -b data/SRR13973705.bam "NC_000017.11:7668421-7687490" > data/SRR13973705_tp53.bam

**Step 5: Add read groups**
- Necessary since read group information not included in FASTQ.
- Read group headers are required for base quality score recalibration.
- Relevant information available in SRR.

In [ ]:
%%bash
# Pre-treatment
gatk AddOrReplaceReadGroups -I data/SRR13973704_tp53.bam -O data/SRR13973704_tp53_RG.bam -RGID SRR13973704 -RGLB plasma_266 -RGPL ILLUMINA -RGSM SRR13973704 -RGPU SRR13973704

# During/post-treatment
gatk AddOrReplaceReadGroups -I data/SRR13973705_tp53.bam -O data/SRR13973705_tp53_RG.bam -RGID SRR13973705 -RGLB plasma_266 -RGPL ILLUMINA -RGSM SRR13973705 -RGPU SRR13973705


**Step 6: Create reference sequence dictionary**
- Necessary for base quality score recalibration.

In [ ]:
%%bash
gatk CreateSequenceDictionary -R data/hg38_ref.fa -O data/hg38_ref.dict

**Step 7: Generate index file for known sites VCF**
- Necessary for base quality score recalibration.
- dbSNP used for known sites.

In [ ]:
%%bash
gatk IndexFeatureFile -I data/GCF_000001405_40.vcf

**Step 8: Generate recalibration table**
- Necessary for base quality score recalibration.

In [ ]:
%%bash
# Pre-treatment
gatk BaseRecalibrator -I data/SRR13973704_tp53_RG.bam -R data/hg38_ref.fa --known-sites data/GCF_000001405_40.vcf -O data/SRR13973704_tp53_recal_data.table

# During/post-treatment
gatk BaseRecalibrator -I data/SRR13973705_tp53_RG.bam -R data/hg38_ref.fa --known-sites data/GCF_000001405_40.vcf -O data/SRR13973705_tp53_recal_data.table


**Step 9: Base quality score recalibration**
- Detects systematic errors made by the sequencing machine when it estimates the accuracy of each base call.
- Short variant calling algorithms rely heavily on the quality score assigned to the individual base calls in each sequence read.
- Scores produced by the machines are subject to various sources of systematic (non-random) technical error, leading to over- or under-estimated base quality scores in the data.

In [ ]:
%%bash
# Pre-treatment
gatk ApplyBQSR -R data/hg38_ref.fa -I data/SRR13973704_tp53_RG.bam --bqsr-recal-file data/SRR13973704_tp53_recal_data.table -O data/SRR13973704_tp53_recal.bam

# During/post-treatment
gatk ApplyBQSR -R data/hg38_ref.fa -I data/SRR13973705_tp53_RG.bam --bqsr-recal-file data/SRR13973705_tp53_recal_data.table -O data/SRR13973705_tp53_recal.bam

# Variant Calling Analysis
Variant calling analysis was conducted in adherence with GATK's best practice workflow for somatic short variant discovery (SNVs + Indels) (https://gatk.broadinstitute.org/hc/en-us/articles/360035894731-Somatic-short-variant-discovery-SNVs-Indels).

**Step 10: Variant calling**
- GATK Mutect2 used as it is HaplotypeCaller equivalent for somatic cells.

In [ ]:
%%bash
# Pre-treatment
gatk Mutect2 -R data/hg38_ref.fa -I data/SRR13973704_tp53_recal.bam -O data/SRR13973704.vcf.gz

# During/post-treatment
gatk Mutect2 -R data/hg38_ref.fa -I data/SRR13973705_tp53_recal.bam -O data/SRR13973705.vcf.gz

**Step 11: Filter variants**
- Computes error probability for each candidate variant.
- Filters are divided into 3 categories: technical artifacts, non-somatic, and sequencing error.
- Optimizes F-score -- the harmonic mean of recall and precision -- as its default thresholding strategy.

In [ ]:
%%bash
# Pre-treatment
gatk FilterMutectCalls -R data/hg38_ref.fa -V data/SRR13973704.vcf.gz -O data/SRR13973704_filtered.vcf.gz

#During/post-treatment
gatk FilterMutectCalls -R data/hg38_ref.fa -V data/SRR13973705.vcf.gz -O data/SRR13973705_filtered.vcf.gz

**Step 12: Generate index file for ClinVar annotation VCF**
- Necessary for SnpSift.
- ClinVar used to for clinical relevance annotation.

In [ ]:
%%bash
tabix -p vcf data/clinvar_20251109.vcf.gz

**Step 13: Rename Chr in variants VCF**
- Necessary to ensure proper mapping between ClinVar and variants for annotation.
- ClinVar uses integer notation to represent chomosomes.
- SRR uses RefSeq notation to represent loci.

In [ ]:
%%bash
# Pre-treatment
bcftools annotate --rename-chrs chr_name_conv.txt data/SRR13973704_filtered.vcf.gz -o data/SRR13973704_filtered_renamed.vcf
bgzip data/SRR13973704_filtered_renamed.vcf

# During/post-treatment
bcftools annotate --rename-chrs chr_name_conv.txt data/SRR13973705_filtered.vcf.gz -o data/SRR13973705_filtered_renamed.vcf
bgzip data/SRR13973705_filtered_renamed.vcf

**Step 14: Annotate Variants**
- Annotations from ClinVar databases

In [ ]:
%%bash
# Pre-treatment
SnpSift annotate data/clinvar_20251109.vcf.gz data/SRR13973704_filtered_renamed.vcf.gz > data/SRR13973704_annotated.vcf

# During/post-treatment
SnpSift annotate data/clinvar_20251109.vcf.gz data/SRR13973705_filtered_renamed.vcf.gz > data/SRR13973705_annotated.vcf

**Step 15: Identify Clinically Relevant Variants**
- Variants considered clinically relevant if found in ClinVar.

In [ ]:
import pandas as pd

# Import VCF files to DataFrame

pre_treatment_variants = pd.read_csv('data/SRR13973704_annotated.vcf', comment='#', sep='\t',
                                     names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SRR13973704'])
post_treatment_variants = pd.read_csv('data/SRR13973705_annotated.vcf', comment='#', sep='\t',
                                      names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SRR13973704'])

# Filter variants based w/ ClinVar Annotation
clinically_relevant_pre_treatment_variants = pre_treatment_variants[pre_treatment_variants['ID'] != '.']
clinically_relevant_post_treatment_variants = post_treatment_variants[post_treatment_variants['ID'] != '.']

# Display clinically relevant mutations
clinically_relevant_pre_treatment_variants
clinically_relevant_post_treatment_variants


# Discussion

The goal of this exercise was to conduct variant calling analysis (VCA) for a specific gene of interest on both pre-treatment and during/post-treatment samples from of a singular patient . Specifically, I conducted VCA for TP53 on both samples from patient CTC360. I was able to identify a decrease in both the total number of variants (79 -> 45), and the number clinically relevant variants (16 -> 11), following chemotherapy treatment with cetuximab-containing regimen. This is consistent with findings from *Circulating tumor DNA sequencing in colorectal cancer patients treated with first-line chemotherapy with anti-EGFR*, which showed significant decreases in Variant Allele Frequency (VAF) following treatment in most patients. Additionally, *Lim et al.* showed that the change of average VAF between the baseline and the first response evaluation had a linear correlation with tumor size changes on CT images. TP53 acts as a tumor suppressor in many tumor types -- inducing growth arrest or apoptosis depending on the physiological circumstances and cell type -- and is involved in cell cycle regulation as a trans-activator that acts to negatively regulate cell division by controlling a set of genes required for this process. Since TP53 is known to confer tumor progression, this result could indicate that response to treatment in this patient may be driven by reduced variation in the TP53 region, and thus, a return to normal function of TP53 in the EGFR pathway. This said, given the data considered in this exercise and the sample size, it is impossible to prove this hypothesis.


When displaying these findings for a presentation, I would display the variant information tables from VCF in a tabular format. Additionally, to clearly show the change in number of variants, I would include a bar plot with counts for total variants and clinically relevant variants, pre and during/post-treatment. I would also do a deeper dive into specific variants of interest, including relevant information from ClinVar and COSMIC. Furthermore, I would include some graphics showing an overview of the EGFR pathway to explain why alterations in this pathway could lead to disease progression.